In [8]:
import pandas as pd
import os

In [ ]:
"hello"

In [59]:
path = os.path.abspath('')
path = path + "/engineered_data_250ms_window80_step8.csv"
df = pd.read_csv(path)

diction = {'Dany':0, 'Felix':1, 'Julian':2, 'Mark':3,'Martin':4,'Michele':5,'Paul':6}

for index,row in df['Participant'].items():
    df.loc[index,'Participant'] = diction[row]

In [60]:
#create train/test set
#should maybe do random runs in the future

X_Train = df[df['Run'] < 4].copy()
X_Train
X_Test = df[df['Run'] == 4].copy()

Y_Train = X_Train['Participant'].copy()

Y_Test = X_Test['Participant'].copy()
X_Test.drop(['Participant'],axis=1,inplace=True)
X_Train.drop(['Participant'],axis=1,inplace=True)
X_Train[X_Train['Path'] == 'circle'] = 1
X_Train
X_Train[X_Train['Path'] == 'straight'] = 0
X_Test[X_Test['Path'] == 'circle'] = 1
X_Train
X_Test[X_Test['Path'] == 'straight'] = 0


In [32]:
from sklearn import svm
#rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train, y_train)
clf = svm.SVC(kernel='rbf',gamma=0.5,C=0.1)
poly = svm.SVC(kernel='poly', degree=10, C=1)
clf.fit(X_Train,Y_Train)
poly.fit(X_Train,Y_Train)
y_pred = clf.predict(X_Test)
y_pred_poly = poly.predict(X_Test)
print(y_pred)

['Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Dany'
 'Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Dany'
 'Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Julian' 'Julian' 'Julian' 'Julian'
 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian'
 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian'
 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian'
 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian'
 'Julian' 'Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Dany'
 'Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Dany' 'Julian'
 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian'
 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian'
 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian'
 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian' 'Julian'
 'Julian' 'Julian' 'Dany' 'Dany' 'Dany' 'Dany' 'Julian' 

In [11]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(Y_Test , y_pred))
print("Accuracy:",metrics.accuracy_score(Y_Test , y_pred_poly))


Accuracy: 0.1951219512195122
Accuracy: 0.1951219512195122


0       0
1       0
2       0
3       0
4       0
       ..
1301    6
1302    6
1303    6
1304    6
1305    6
Name: Participant, Length: 1019, dtype: int64

In [62]:

X_Train.dtypes
X_Train.drop('Path',axis=1,inplace=True)
X_Test.drop('Path',axis=1,inplace=True)

In [68]:


from xgboost import XGBClassifier
model = XGBClassifier(tree_method = 'hist',objective='multi:softprob')
model.fit(X_Train,Y_Train)
y_pred = model.predict(X_Test)
Y_Test = Y_Test.astype('int')
print("Accuracy:",metrics.accuracy_score(Y_Test , y_pred))


Accuracy: 0.1951219512195122
